In [784]:
import cv2
import numpy as np
from random import randint

In [785]:
def resize(img, times):
    new_width = int(img.shape[1] * times)
    new_height = int(img.shape[0] * times)
    return cv2.resize( img, (new_width, new_height), cv2.INTER_LANCZOS4)

In [786]:
def attach_background(img, background):
    
    bg_height, bg_width, _ = background.shape
    img_height, img_width, _ = img.shape
    
    assert( bg_height > img_height)
    assert( bg_width > img_width)
    
    offset_var_x = (bg_width - img_width)   // 3
    offset_var_y = (bg_height - img_height) // 3 
    x_offset = (bg_width - img_width)   // 2 + randint(-offset_var_x, offset_var_x)
    y_offset = (bg_height - img_height) // 2 + randint(-offset_var_y, offset_var_y)
    res = background.copy()
    res[y_offset:y_offset+img_height, x_offset:x_offset+img_width] = img

    return res

In [800]:
def add_noise(img):
    mean = np.random.normal(0.2, 0.05)
    var  = np.abs(np.random.normal(0.07, 0.05))
    gauss_noise = (np.random.normal(mean, var, img.shape) * 255).astype("uint8")
    return cv2.add(img, gauss_noise)

In [788]:
def random_line(img):
    position = randint(0, img.shape[1])
    width = randint(2, 4)
    res = cv2.line(img, (position,0), (position,img.shape[0]), (50, 50, 50), width) 
    return res

In [789]:
def perspective_transform(img):
    img_height, img_width, _ = img.shape

    vec_var = 0.15
    vec1 = np.array([0, -1, 0]) + np.random.uniform(-vec_var, vec_var, 3)
    vec2 = np.array([1, 0, 0])  + np.random.uniform(-vec_var, vec_var, 3)
    vec1 = vec1 / np.linalg.norm(vec1) * (img_height / 2)
    vec2 = vec2 / np.linalg.norm(vec2) * (img_width  / 2)
    new_centre = np.array([img_width  / 2, img_height / 2, 0])
    left_up    = (new_centre + vec1 - vec2)[:2]
    left_down  = (new_centre - vec1 - vec2)[:2]
    right_up   = (new_centre + vec1 + vec2)[:2]
    right_down = (new_centre - vec1 + vec2)[:2]

    pts1 = np.float32([[0, 0],          [img_width, 0],
                       [0, img_height], [img_width, img_height]])
    pts2 = np.float32([left_up,   right_up,
                       left_down, right_down])

    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    result = cv2.warpPerspective(img, matrix, (img_width, img_height))

    left_x  = int(max(left_up[0], left_down[0]))
    right_x = int(min(right_up[0], right_down[0]))
    up_y    = int(max(left_up[1], right_up[1]))
    down_y  = int(min(left_down[1], right_down[1]))

    result = result[up_y:down_y, left_x:right_x]

    return result

In [790]:
def change_light(img):
    img_height, img_width, _ = img.shape
    centre_x = randint( int(-img_width * 0.2), int(img_width * 1.2))
    centre_y = randint( int(-img_height * 0.2), int(img_height * 1.2))
    light_centre = np.array([centre_x, centre_y])
    max_val = 2
    speed = -0.0015
    for y in range(0, img_height):
         for x in range(0, img_width):
            dist = np.linalg.norm(np.array([x, y]) - light_centre)
            brightness = speed * dist + max_val
            res = np.clip(img[y, x] * brightness, 0, 255)
            img[y, x] = res
    return img

In [791]:
def generate(img, background):
    background = resize(background, 2.3)
    img = resize(img, 0.25)

    img = cv2.GaussianBlur(img,(3,3), 0.1)

    if randint(0, 100) < 70:
        img = add_noise(img)

    if randint(0, 100) < 30:
        img = random_line(img)

    res = attach_background(img, background)
    res = perspective_transform(res)
    
    if randint(0, 100) < 40:
        res = change_light(res)
    
    return res

In [802]:
def generate_loop():
    mul_num = 10
    res_num = 127
    for img_id in range(1, 1):
        for _ in range(mul_num):
            img = cv2.imread(f'./docs/{img_id}.png')
            bg_id = randint(1, 1)
            background = cv2.imread(f'./background/{bg_id}.jpg') 
            res = generate(img, background)
            cv2.imwrite(f'./generated_photos/{res_num}.png', res)
            res_num += 1

In [803]:
generate_loop()

KeyboardInterrupt: 